In [1]:
import numpy as np

In [2]:
#F is dependant on x and m.
#f_n is F with the values of x and m at time n

In [19]:
class Body():
    def __init__(self, x:float, y:float, m:float) -> None:
        self.init_position:np.ndarray = np.array([x,y])
        self.position:np.ndarray = np.array([x,y])
        self.mass:float = m 
        self.trajectory:list[np.ndarray] = [self.init_position]

    def update_pos(self, new_pos:np.ndarray):
        self.position = new_pos
        self.trajectory.append(self.position)

    def f_x_y(self, otherbody:'Body'):
        """
        Force of function from x to y.
        """
        G = 6.67e-11
        r = otherbody.position - self.position
        distance = np.linalg.norm(r)
        r = r/distance
        return G * (otherbody.mass * self.mass) * r / (distance ** 2)

In [16]:
class Simulation():
    def __init__(self, bodies:list[Body], t_max:int=10, t_step:float=0.1, step:int=1, alphas:list=[-1,1], betas:list=[1]):
        self.bodies:list[Body] = bodies
        self.t_max:int = t_max
        self.dt:float = t_step
        self.step:int = step
        self.alphas:list = alphas
        self.betas:list = betas

    def run(self):
        for t in np.arange(0, self.t_max, self.dt):
            print(t)

In [11]:
f_x_y(1,2,np.array([1,2]),np.array([2,3]))

array([4.71640223e-11, 4.71640223e-11])

In [18]:
sim = Simulation([Body(0,0,0)])
sim.run()

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
1.0
1.1
1.2000000000000002
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.8
1.9000000000000001
2.0
2.1
2.2
2.3000000000000003
2.4000000000000004
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5
3.6
3.7
3.8000000000000003
3.9000000000000004
4.0
4.1000000000000005
4.2
4.3
4.4
4.5
4.6000000000000005
4.7
4.800000000000001
4.9
5.0
5.1000000000000005
5.2
5.300000000000001
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.9
6.0
6.1000000000000005
6.2
6.300000000000001
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.0
7.1000000000000005
7.2
7.300000000000001
7.4
7.5
7.6000000000000005
7.7
7.800000000000001
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.700000000000001
8.8
8.9
9.0
9.1
9.200000000000001
9.3
9.4
9.5
9.600000000000001
9.700000000000001
9.8
9.9
